# Review Helpfulness Classification - ML Models
* Model Baselines - RF(Random Forest), SVM(Support Vector Machine)
* Dataset - Amazon(CD&Vinyls, Toys and Games)
* Features - LIWC Contents(Linguistic, Psychological), Readability, Subjectivity, Sentiment

In [ ]:
!pip install textstat

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install contractions

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import re
import nltk
import textstat
import contractions
import pandas as pd
import numpy as np
from textblob import TextBlob
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from dateutil import relativedelta
from sklearn.preprocessing import MinMaxScaler
from nltk import sent_tokenize, word_tokenize
from sklearn.linear_model import LogisticRegression
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from tensorflow.keras.preprocessing.text import Tokenizer
from collections import Counter
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [ ]:
amazon = pd.read_csv('/content/drive/MyDrive/seminar/bert_review_helpfulness/datasets/datasets/Overall_Amazon.csv')
amazon = amazon[['Item_Title', 'Item_Rating','Item_Review_Count', 'User_Link',
                 'Review_Rating', 'Review_Text', 'Review_Helpfulness',
                 'User_Helpful_Votes', 'User_Reviews_Count', 'User_Ranking', 'Product URL']]
print(amazon.shape)
amazon.head()

(1186795, 11)


,Item_Title,Item_Rating,Item_Review_Count,User_Link,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking,Product URL
0,All American Christmas,4.4,30.0,https://www.amazon.com//gp/profile/amzn1.accou...,5.0,Very heartwarming and beautiful close-to-the-h...,33.0,71.0,20.0,4328158.0,https://www.amazon.com//All-American-Christmas...
1,All American Christmas,4.4,30.0,https://www.amazon.com//gp/profile/amzn1.accou...,5.0,I liked everything about it. It arrived on ti...,24.0,58.0,74.0,959160.0,https://www.amazon.com//All-American-Christmas...
2,All American Christmas,4.4,30.0,https://www.amazon.com//gp/profile/amzn1.accou...,5.0,Love getting to read about holiday traditions ...,18.0,27.0,8.0,14906136.0,https://www.amazon.com//All-American-Christmas...
3,All American Christmas,4.4,30.0,https://www.amazon.com//gp/profile/amzn1.accou...,5.0,Can't wait to read and see all the recipes tha...,10.0,30.0,57.0,5168935.0,https://www.amazon.com//All-American-Christmas...
4,All American Christmas,4.4,30.0,https://www.amazon.com//gp/profile/amzn1.accou...,5.0,Love this book. Very reminiscent of my ow chil...,12.0,19.0,3.0,28735105.0,https://www.amazon.com//All-American-Christmas...


In [ ]:
amazon.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1186795 entries, 0 to 1186794
Data columns (total 11 columns):
 #   Column              Non-Null Count    Dtype  
---  ------              --------------    -----  
 0   Item_Title          1186795 non-null  object 
 1   Item_Rating         1118130 non-null  float64
 2   Item_Review_Count   1118130 non-null  float64
 3   User_Link           1186725 non-null  object 
 4   Review_Rating       1186725 non-null  float64
 5   Review_Text         1185951 non-null  object 
 6   Review_Helpfulness  1186725 non-null  float64
 7   User_Helpful_Votes  1065517 non-null  float64
 8   User_Reviews_Count  1065506 non-null  float64
 9   User_Ranking        1064609 non-null  float64
 10  Product URL         1186795 non-null  object 
dtypes: float64(7), object(4)
memory usage: 99.6+ MB


In [ ]:
# 중복제거 및 결측치 컬럼 제거
col = ['User_Link', 'Product URL', 'Review_Text']
amazon = amazon.drop_duplicates(subset=col, ignore_index=True)
amazon = amazon.drop(columns=['User_Link', 'Product URL'])
amazon = amazon.dropna()
print(amazon.shape)
amazon.head()

(986605, 9)


,Item_Title,Item_Rating,Item_Review_Count,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking
0,All American Christmas,4.4,30.0,5.0,Very heartwarming and beautiful close-to-the-h...,33.0,71.0,20.0,4328158.0
1,All American Christmas,4.4,30.0,5.0,I liked everything about it. It arrived on ti...,24.0,58.0,74.0,959160.0
2,All American Christmas,4.4,30.0,5.0,Love getting to read about holiday traditions ...,18.0,27.0,8.0,14906136.0
3,All American Christmas,4.4,30.0,5.0,Can't wait to read and see all the recipes tha...,10.0,30.0,57.0,5168935.0
4,All American Christmas,4.4,30.0,5.0,Love this book. Very reminiscent of my ow chil...,12.0,19.0,3.0,28735105.0


In [ ]:
# Review Length 생성
amazon['Review_Length'] = amazon['Review_Text'].apply(lambda x: len(x.split(' ')))
amazon.head()

,Item_Title,Item_Rating,Item_Review_Count,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking,Review_Length
0,All American Christmas,4.4,30.0,5.0,Very heartwarming and beautiful close-to-the-h...,33.0,71.0,20.0,4328158.0,14
1,All American Christmas,4.4,30.0,5.0,I liked everything about it. It arrived on ti...,24.0,58.0,74.0,959160.0,17
2,All American Christmas,4.4,30.0,5.0,Love getting to read about holiday traditions ...,18.0,27.0,8.0,14906136.0,19
3,All American Christmas,4.4,30.0,5.0,Can't wait to read and see all the recipes tha...,10.0,30.0,57.0,5168935.0,31
4,All American Christmas,4.4,30.0,5.0,Love this book. Very reminiscent of my ow chil...,12.0,19.0,3.0,28735105.0,29


In [ ]:
# User_Ranking - 랭킹이 1만등 이내인 경우 1을, 1만등 밖인 경우 0 부여
def user_ranking(x):
  if x <= 10000:
    x = 1

  else: x = 0
  return x

amazon['User_Ranking'] = amazon['User_Ranking'].apply(lambda x: user_ranking(x))
amazon.head()

,Item_Title,Item_Rating,Item_Review_Count,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking,Review_Length
0,All American Christmas,4.4,30.0,5.0,Very heartwarming and beautiful close-to-the-h...,33.0,71.0,20.0,0,14
1,All American Christmas,4.4,30.0,5.0,I liked everything about it. It arrived on ti...,24.0,58.0,74.0,0,17
2,All American Christmas,4.4,30.0,5.0,Love getting to read about holiday traditions ...,18.0,27.0,8.0,0,19
3,All American Christmas,4.4,30.0,5.0,Can't wait to read and see all the recipes tha...,10.0,30.0,57.0,0,31
4,All American Christmas,4.4,30.0,5.0,Love this book. Very reminiscent of my ow chil...,12.0,19.0,3.0,0,29


In [ ]:
def label_convert(x):
  '''
  Review_Helpfulness 구분
  Review_Helpfulness >= 4면 1
  Review_Helpfulness == 0이면 0
  나머지 1,2,3의 경우는 배제
  '''
  if x >= 4:
    x = 1
  elif x == 0:
    x = 0
  else : x = 'drop'
  return x

amazon['Review_Helpfulness'] = amazon['Review_Helpfulness'].apply(lambda x: label_convert(int(x)))
amazon = amazon[amazon['Review_Helpfulness'] != 'drop']
amazon['Review_Helpfulness'].value_counts()

0    930888
1     27410
Name: Review_Helpfulness, dtype: int64

In [ ]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

def clean_text(review_text):
    review_text = BeautifulSoup(review_text , "html.parser").get_text()
    review_text = contractions.fix(review_text)
    review_text = re.sub('[^a-zA-Z]', ' ', review_text)
    review_text = str(review_text).lower()
    review_text = word_tokenize(review_text)
    review_text = [item for item in review_text if item not in stop_words]
    review_text = [lemmatizer.lemmatize(word = w, pos='v') for w in review_text]
    review_text = [i for i in review_text if len(i) > 2]
    review_text = ' '.join(review_text)
    return review_text

amazon['Cleaned_Review_Text']= amazon['Review_Text'].apply(lambda x: clean_text(x))
amazon.head()

<ipython-input-46-13bc372fce55>:5: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  review_text = BeautifulSoup(review_text , "html.parser").get_text()
<ipython-input-46-13bc372fce55>:5: MarkupResemblesLocatorWarning: The input looks more like a URL than markup. You may want to use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  review_text = BeautifulSoup(review_text , "html.parser").get_text()


,Item_Title,Item_Rating,Item_Review_Count,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking,Review_Length,Cleaned_Review_Text
0,All American Christmas,4.4,30.0,5.0,Very heartwarming and beautiful close-to-the-h...,1,71.0,20.0,0,14,heartwarming beautiful close heart stories bri...
1,All American Christmas,4.4,30.0,5.0,I liked everything about it. It arrived on ti...,1,58.0,74.0,0,17,like everything arrive time make perfect gift
2,All American Christmas,4.4,30.0,5.0,Love getting to read about holiday traditions ...,1,27.0,8.0,0,19,love get read holiday traditions favorite peop...
3,All American Christmas,4.4,30.0,5.0,Can't wait to read and see all the recipes tha...,1,30.0,57.0,0,31,wait read see recipes fox news group put book ...
4,All American Christmas,4.4,30.0,5.0,Love this book. Very reminiscent of my ow chil...,1,19.0,3.0,0,29,love book reminiscent childhood memories nicel...


In [ ]:
def get_subjectivity(x):
  '''
  subjectivity - subjectivity 컬럼 생성
  textblob 패키지 사용
  '''
  return TextBlob(x).sentiment.subjectivity

def get_polarity(x):
  '''
  get_polarity - Sentiment 컬럼 생성
  textblob 패키지 사용
  '''
  return TextBlob(x).sentiment.polarity

def get_readability(x):
  '''
  get_readability - readability 컬럼 생성
  textstat의 flesch_reading_ease 패키지 사용
  '''
  return textstat.flesch_reading_ease(x)

amazon['subjectivity'] = amazon['Review_Text'].apply(lambda x: get_subjectivity(x))
amazon['sentiment'] = amazon['Review_Text'].apply(lambda x: get_polarity(x))
amazon['readability'] = amazon['Review_Text'].apply(lambda x: get_readability(x))
amazon.head()

,Item_Title,Item_Rating,Item_Review_Count,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking,Review_Length,Cleaned_Review_Text,subjectivity,sentiment,readability
0,All American Christmas,4.4,30.0,5.0,Very heartwarming and beautiful close-to-the-h...,1,71.0,20.0,0,14,heartwarming beautiful close heart stories bri...,0.650000,0.525000,39.50
1,All American Christmas,4.4,30.0,5.0,I liked everything about it. It arrived on ti...,1,58.0,74.0,0,17,like everything arrive time make perfect gift,0.900000,0.800000,88.74
2,All American Christmas,4.4,30.0,5.0,Love getting to read about holiday traditions ...,1,27.0,8.0,0,19,love get read holiday traditions favorite peop...,0.533333,0.333333,69.11
3,All American Christmas,4.4,30.0,5.0,Can't wait to read and see all the recipes tha...,1,30.0,57.0,0,31,wait read see recipes fox news group put book ...,0.666667,0.641667,103.93
4,All American Christmas,4.4,30.0,5.0,Love this book. Very reminiscent of my ow chil...,1,19.0,3.0,0,29,love book reminiscent childhood memories nicel...,0.615909,0.287879,64.57


In [ ]:
# 등장 빈도수(3회) 이상 단어만 사용
word_counts = Counter(' '.join(amazon['Cleaned_Review_Text']).split())
freq_threshold = 3
filtered_words = [word for word in word_counts if word_counts[word] >= freq_threshold]

amazon['Freq_Review_Text'] = amazon['Cleaned_Review_Text'].apply(lambda x: ' '.join([word for word in x.split() if word in filtered_words]))
amazon.head()

,Item_Title,Item_Rating,Item_Review_Count,Review_Rating,Review_Text,Review_Helpfulness,User_Helpful_Votes,User_Reviews_Count,User_Ranking,Review_Length,Cleaned_Review_Text,subjectivity,sentiment,readability,Freq_Review_Text
0,All American Christmas,4.4,30.0,5.0,Very heartwarming and beautiful close-to-the-h...,1,71.0,20.0,0,14,heartwarming beautiful close heart stories bri...,0.650000,0.525000,39.50,heartwarming beautiful close heart stories bri...
1,All American Christmas,4.4,30.0,5.0,I liked everything about it. It arrived on ti...,1,58.0,74.0,0,17,like everything arrive time make perfect gift,0.900000,0.800000,88.74,like everything arrive time make perfect gift
2,All American Christmas,4.4,30.0,5.0,Love getting to read about holiday traditions ...,1,27.0,8.0,0,19,love get read holiday traditions favorite peop...,0.533333,0.333333,69.11,love get read holiday traditions favorite peop...
3,All American Christmas,4.4,30.0,5.0,Can't wait to read and see all the recipes tha...,1,30.0,57.0,0,31,wait read see recipes fox news group put book ...,0.666667,0.641667,103.93,wait read see recipes fox news group put book ...
4,All American Christmas,4.4,30.0,5.0,Love this book. Very reminiscent of my ow chil...,1,19.0,3.0,0,29,love book reminiscent childhood memories nicel...,0.615909,0.287879,64.57,love book reminiscent childhood memories nicel...


In [ ]:
amazon.to_csv('amazon_prepro_4over.csv', index=False)